In [1]:
import os
import numpy as np
from PIL import Image

from tqdm import notebook
def tqdm(x, **kargs):
    return notebook.tqdm(x, leave=False, **kargs)

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset
import torchvision.transforms as transforms
import torchvision.models as models

from efficientnet_pytorch import EfficientNet

from sklearn.metrics import f1_score, precision_recall_fscore_support

In [2]:
class customDataset(Dataset):
    def __init__(self, datatype, transform):
        ##############################################
        ### Initialize paths, transforms, and so on
        ##############################################
        self.transform = transform

        filename = open('./crawling/{}_list'.format(datatype), 'r')
        info = filename.readlines()
        self.images = [row.split(',')[0] for row in info]
        self.labels = [int(row.split(',')[1].strip()) for row in info]
        
        assert len(self.images) == len(self.labels), 'mismatched length!'
        print("image shape: {}, label shape: {}".format(len(self.images), len(self.labels)))
        
        
    def __getitem__(self, index):
        ##############################################
        # 1. Read from file (using numpy.fromfile, PIL.Image.open)
        # 2. Preprocess the data (torchvision.Transform).
        # 3. Return the data (e.g. image and label)
        ##############################################
        
        image = Image.open(self.images[index]).convert('RGB')
        label = self.labels[index]
        
        if self.transform is not None:
            image = self.transform(image)
        return image, label
        
    def __len__(self):
        ##############################################
        ### Indicate the total size of the dataset
        ##############################################
        return len(self.images)

In [3]:
batch_size = 32
num_workers = 8
# min (100, 100), max (5792, 8688)
resize_size = (224, 224)
MODEL = 'efficientnet-b4'

In [4]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(resize_size),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.5, contrast=0.5),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ]),
    'test': transforms.Compose([
        transforms.Resize(resize_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ]),
}

In [5]:
trainset = customDataset(datatype='train',
                         transform=data_transforms['train'])
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, num_workers=num_workers, shuffle=True)

image shape: 45242, label shape: 45242


In [6]:
testset = customDataset(datatype='test',
                        transform=data_transforms['test'])
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, num_workers=num_workers, shuffle=False)

image shape: 11309, label shape: 11309


In [7]:
for imgs, lbls in trainloader:
    print('Size of image:', imgs.size())  
    print('Type of image:', imgs.dtype)
    print('Size of label:', lbls.size())  
    print('Type of label:', lbls.dtype)
    break

Size of image: torch.Size([32, 3, 224, 224])
Type of image: torch.float32
Size of label: torch.Size([32])
Type of label: torch.int64


In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        # self.pretrained = models.resnet50(pretrained=True)
        # self.pretrained = models.densenet161(pretrained=True)
        self.pretrained = EfficientNet.from_pretrained('efficientnet-b4')

        # freeze pretrained models
        # ct = 0
        for child in self.pretrained.children():
            # ct += 1
            # if ct < 5:
            for param in child.parameters():
                param.requires_grad = False


        self.relu = nn.ReLU()
        self.fc = nn.Linear(1000, 2)

    def forward(self, x):
        x = self.pretrained(x)
        x = self.relu(x)
        x = self.fc(x)
        return x
net = Net()
net.to(device)

Loaded pretrained weights for efficientnet-b4


ernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          28, 672, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_project_conv): Conv2dStaticSamePadding(
          672, 160, kernel_size=(1, 1), stride=(1, 1), bias=False
          (static_padding): Identity()
        )
        (_bn2): BatchNorm2d(160, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_swish): MemoryEfficientSwish()
      )
      (17): MBConvBlock(
        (_expand_conv): Conv2dStaticSamePadding(
          160, 960, kernel_size=(1, 1), stride=(1, 1), bias=False
          (static_padding): Identity()
        )
        (_bn0): BatchNorm2d(960, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_depthwise_conv): Conv2dStaticSamePadding(
          960, 960, kernel_size=(5, 5), stride=(1, 1), groups=960, bias=False
        

In [10]:
# positive: 15433, negative: 32895
class_weights = torch.FloatTensor([1, 2.6646]).cuda()
criterion = nn.CrossEntropyLoss(weight=class_weights)
epochs = 5
size = 45242 // batch_size

In [10]:
### First
LR = 1e-2
# optimizer = optim.Adam(net.parameters(), lr=1e-3)
optimizer = optim.SGD(net.parameters(), lr=LR)

for epoch in tqdm(range(epochs), desc='Epochs: '):  # loop over the dataset multiple times
    print("================== EPOCH INFO: {} ==================".format(epoch + 1))
    net.train()     # Train mode
    running_loss = 0.0
    for i, data in enumerate(tqdm(trainloader, desc='Progress:'), 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        labels = labels.view(-1)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)

        # print(data[0].shape, data[1].shape)
        # print(inputs.shape)
        # print(labels.shape, outputs.shape)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 500 == 0 and i != 0:    # print every size mini-batches
            _, y_pred_tag = torch.max(outputs, 1)
            correct_results_sum = (y_pred_tag == labels).sum().float()
            acc = correct_results_sum / labels.shape[0]

            y_pred_np = y_pred_tag.cpu().detach().numpy()
            y_true_np = labels.cpu().detach().numpy()
            print("==============================================")
            print("Batch acc = {} ({} / {})".format(acc, correct_results_sum, labels.shape[0]))
            print("Batch f1-score = {}".format(f1_score(y_true_np, y_pred_np, average='binary')))
            print(precision_recall_fscore_support(y_true_np, y_pred_np, average='binary'))

            print("Prediction distribution:")
            print(np.unique(y_pred_np, return_counts=True), np.unique(y_true_np, return_counts=True))
            # print(y_pred_np, y_true_np)

            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 500))
            running_loss = 0.0

    # Eval mode
    net.eval()
    # Training score
    y_pred_train = np.array([])
    y_true_train = np.array([])
    with torch.no_grad():
        for data_test in tqdm(trainloader):
            images, labels = data_test[0].to(device), data_test[1].to(device)
            outputs = net(images)

            _, y_pred_tag = torch.max(outputs, 1)
            y_pred_train = np.hstack([y_pred_train, y_pred_tag.cpu().detach().numpy()])

            y_true_train = np.hstack([y_true_train, labels.cpu().detach().numpy()])

    precision_train, recall_train, f1_train, support_train = precision_recall_fscore_support(y_true_train, y_pred_train, average='binary')

    print()
    print("Training prediction:")
    print(np.unique(y_pred_train, return_counts=True), np.unique(y_true_train, return_counts=True))
    print(y_pred_train[0:10], y_true_train[0:10])
    print("F1 of model on train set: ", f1_score(y_true_train, y_pred_train, average='binary'))
    print("Precision: {}, Recall: {}, F1: {}".format(precision_train, recall_train, f1_train))

    # Testing score
    y_pred_test = np.array([])
    y_true_test = np.array([])
    with torch.no_grad():
        for data_test in tqdm(testloader):
            images, labels = data_test[0].to(device), data_test[1].to(device)
            outputs = net(images)

            _, y_pred_tag = torch.max(outputs, 1)
            y_pred_test = np.hstack([y_pred_test, y_pred_tag.cpu().detach().numpy()])
                
            y_true_test = np.hstack([y_true_test, labels.cpu().detach().numpy()])
    
    precision_test, recall_test, f1_test, support_test = precision_recall_fscore_support(y_true_test, y_pred_test, average='binary')

    print("Testing prediction:")
    print(np.unique(y_pred_test, return_counts=True), np.unique(y_true_test, return_counts=True))
    print(y_pred_test[0:10], y_true_test[0:10])
    print("F1 of my model on test set: ", f1_score(y_true_test, y_pred_test, average='binary'))
    print("Precision: {}, Recall: {}, F1: {}".format(precision_test, recall_test, f1_test))


    # record precision, recall, f1-score into file
    with open('record/train_loss', 'a') as f_train, open('record/test_loss', 'a') as f_test:
        f_train.write(str(precision_train) + ', ' + str(recall_train) + ', ' + str(f1_train) + '\n')
        f_test.write(str(precision_test) + ', ' + str(recall_test) + ', ' + str(f1_test) + '\n')

    # save model for every epoch
    torch.save(net.state_dict(), './model/{}_{}_{}_{}_{}'.format(MODEL, LR, epoch, f1_train, f1_test))

================== EPOCH INFO: 1 ==================


Batch acc = 0.3125 (10.0 / 32)
Batch f1-score = 0.26666666666666666
(0.16666666666666666, 0.6666666666666666, 0.26666666666666666, None)
Prediction distribution:
(array([0, 1]), array([ 8, 24])) (array([0, 1]), array([26,  6]))
[1 0 0 1 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 0 1 0 1 1 1] [0 0 1 1 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
[1,   501] loss: 0.738
Batch acc = 0.6875 (22.0 / 32)
Batch f1-score = 0.0
(0.0, 0.0, 0.0, None)
Prediction distribution:
(array([0]), array([32])) (array([0, 1]), array([22, 10]))
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] [0 0 1 1 1 0 0 0 0 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0]
[1,  1001] loss: 0.734



Training prediction:
(array([0., 1.]), array([26043, 19199])) (array([0., 1.]), array([32895, 12347]))
[1. 1. 0. 1. 0. 1. 0. 1. 1. 1.] [0. 0. 0. 0. 0. 0. 0. 1. 1. 0.]
F1 of model on train set:  0.3637228174728967
(0.298817646752435, 0.46464728274074674, 0.3637228174728967, None)
(0.5225033209379424, 0.5277028783364777, 0.5115807748499914, None)


Testing prediction:
(array([0., 1.]), array([5929, 5380])) (array([0., 1.]), array([8223, 3086]))
[1. 1. 1. 1. 0. 0. 1. 1. 0. 0.] [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
F1 of my model on test set:  0.3649893692416726
(0.2871747211895911, 0.5006480881399871, 0.3649893692416726, None)
(0.5136328994715033, 0.5171366428782144, 0.49255686664457854, None)
================== EPOCH INFO: 2 ==================


Batch acc = 0.46875 (15.0 / 32)
Batch f1-score = 0.45161290322580644
(0.4117647058823529, 0.5, 0.45161290322580644, None)
Prediction distribution:
(array([0, 1]), array([15, 17])) (array([0, 1]), array([18, 14]))
[1 1 1 0 0 0 1 1 1 0 1 0 0 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 1 0 0 1] [0 1 1 1 1 1 0 1 0 0 1 1 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 1 0 0 0 1]
[2,   501] loss: 0.733
Batch acc = 0.4375 (14.0 / 32)
Batch f1-score = 0.3076923076923077
(0.21052631578947367, 0.5714285714285714, 0.3076923076923077, None)
Prediction distribution:
(array([0, 1]), array([13, 19])) (array([0, 1]), array([25,  7]))
[0 0 1 1 1 1 0 1 1 0 1 1 0 0 1 1 0 1 1 1 0 1 1 0 1 1 0 1 0 1 0 0] [0 1 0 0 0 0 0 1 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0]
[2,  1001] loss: 0.734


...


Training prediction:
(array([0., 1.]), array([23769, 21473])) (array([0., 1.]), array([32895, 12347]))
[1. 0. 0. 0. 0. 0. 0. 0. 1. 1.] [0. 0. 0. 1. 1. 1. 0. 0. 0. 1.]
F1 of model on train set:  0.3744529863985807
(0.294881944767848, 0.5128371264274723, 0.3744529863985807, None)
(0.5209106177202865, 0.5262772043446071, 0.5005471200523188, None)


Testing prediction:
(array([0., 1.]), array([6589, 4720])) (array([0., 1.]), array([8223, 3086]))
[1. 1. 0. 0. 0. 0. 1. 1. 1. 0.] [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
F1 of my model on test set:  0.345119139123751
(0.2853813559322034, 0.4364873622812703, 0.345119139123751, None)
(0.5107283164544913, 0.513148217198035, 0.4999967826323588, None)
================== EPOCH INFO: 5 ==================


Batch acc = 0.46875 (15.0 / 32)
Batch f1-score = 0.37037037037037035
(0.2631578947368421, 0.625, 0.37037037037037035, None)
Prediction distribution:
(array([0, 1]), array([13, 19])) (array([0, 1]), array([24,  8]))
[1 1 0 0 0 1 0 1 1 1 1 1 0 1 0 1 0 1 1 0 1 0 1 1 0 1 1 0 0 1 0 1] [0 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 0 0 0]
[5,   501] loss: 0.731
Batch acc = 0.34375 (11.0 / 32)
Batch f1-score = 0.43243243243243246
(0.3076923076923077, 0.7272727272727273, 0.43243243243243246, None)
Prediction distribution:
(array([0, 1]), array([ 6, 26])) (array([0, 1]), array([21, 11]))
[1 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1] [0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 1 0 0 0 1 1 1 1 0]
[5,  1001] loss: 0.731



Training prediction:
(array([0., 1.]), array([29791, 15451])) (array([0., 1.]), array([32895, 12347]))
[1. 1. 1. 1. 1. 0. 0. 0. 1. 0.] [0. 0. 0. 0. 0. 1. 0. 0. 1. 0.]
F1 of model on train set:  0.33326138571120223
(0.2997864215908356, 0.3751518587511136, 0.33326138571120223, None)
(0.5204077957371787, 0.5231284449554322, 0.518798641041799, None)


Testing prediction:
(array([0., 1.]), array([7522, 3787])) (array([0., 1.]), array([8223, 3086]))
[1. 0. 1. 0. 0. 0. 0. 0. 0. 0.] [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
F1 of my model on test set:  0.318929143023425
(0.28941114338526536, 0.35515230071289694, 0.318929143023425, None)
(0.5124269223972325, 0.5139497366388271, 0.5108142063164124, None)


In [11]:
### Second
LR = 1e-3
# optimizer = optim.Adam(net.parameters(), lr=1e-3)
optimizer = optim.SGD(net.parameters(), lr=LR)

for epoch in tqdm(range(epochs), desc='Epochs: '):  # loop over the dataset multiple times
    print("================== EPOCH INFO: {} ==================".format(epoch + 1))
    net.train()     # Train mode
    running_loss = 0.0
    for i, data in enumerate(tqdm(trainloader, desc='Progress:'), 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        labels = labels.view(-1)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        # labels = labels.type_as(outputs)

        # print(data[0].shape, data[1].shape)
        # print(inputs.shape)
        # print(labels.shape, outputs.shape)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 500 == 0 and i != 0:    # print every size mini-batches
            # y_pred_tag = torch.round(torch.sigmoid(outputs))
            # correct_results_sum = (y_pred_tag == labels).sum().float()
            # acc = correct_results_sum / labels.shape[0]
            _, y_pred_tag = torch.max(outputs, 1)
            correct_results_sum = (y_pred_tag == labels).sum().float()
            acc = correct_results_sum / labels.shape[0]

            y_pred_np = y_pred_tag.cpu().detach().numpy()
            y_true_np = labels.cpu().detach().numpy()
            print("==============================================")
            print("Batch acc = {} ({} / {})".format(acc, correct_results_sum, labels.shape[0]))
            print("Batch f1-score = {}".format(f1_score(y_true_np, y_pred_np, average='binary')))
            print(precision_recall_fscore_support(y_true_np, y_pred_np, average='binary'))

            print("Prediction distribution:")
            print(np.unique(y_pred_np, return_counts=True), np.unique(y_true_np, return_counts=True))
            print(y_pred_np, y_true_np)
            # print(outputs)

            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 500))
            running_loss = 0.0

    # Eval mode
    net.eval()
    # Training score
    y_pred_train = np.array([])
    y_true_train = np.array([])
    with torch.no_grad():
        for data_test in tqdm(trainloader):
            images, labels = data_test[0].to(device), data_test[1].to(device)
            outputs = net(images)
            # outputs = torch.round(torch.sigmoid(outputs))
            _, y_pred_tag = torch.max(outputs, 1)
            y_pred_train = np.hstack([y_pred_train, y_pred_tag.cpu().detach().numpy()])

            # for pred_class in outputs.cpu().detach().numpy():
            #     y_pred_train = np.hstack([y_pred_train, pred_class])

            y_true_train = np.hstack([y_true_train, labels.cpu().detach().numpy()])

    print()
    print("Training prediction:")
    print(np.unique(y_pred_train, return_counts=True), np.unique(y_true_train, return_counts=True))
    print(y_pred_train[0:10], y_true_train[0:10])
    print("F1 of model on train set: ", f1_score(y_true_train, y_pred_train))
    print(precision_recall_fscore_support(y_true_train, y_pred_train, average='binary'))

    # Testing score
    y_pred_test = np.array([])
    y_true_test = np.array([])
    with torch.no_grad():
        for data_test in tqdm(testloader):
            images, labels = data_test[0].to(device), data_test[1].to(device)
            outputs = net(images)
            # outputs = torch.round(torch.sigmoid(outputs))

            _, y_pred_tag = torch.max(outputs, 1)
            y_pred_test = np.hstack([y_pred_test, y_pred_tag.cpu().detach().numpy()])

            # for pred_class in outputs.cpu().detach().numpy():
            #     y_pred_test = np.hstack([y_pred_test, pred_class])
                
            y_true_test = np.hstack([y_true_test, labels.cpu().detach().numpy()])
    
    print("Testing prediction:")
    print(np.unique(y_pred_test, return_counts=True), np.unique(y_true_test, return_counts=True))
    print(y_pred_test[0:10], y_true_test[0:10])
    print("F1 of my model on test set: ", f1_score(y_true_test, y_pred_test))
    print(precision_recall_fscore_support(y_true_test, y_pred_test, average='binary'))

    precision_train, recall_train, f1_train, support_train = precision_recall_fscore_support(y_true_train, y_pred_train, average='binary')
    torch.save(net.state_dict(), './model/{}_{}_{}_{}'.format(MODEL, LR, epoch, f1_train))

================== EPOCH INFO: 1 ==================


Batch acc = 0.6875 (22.0 / 32)
Batch f1-score = 0.4444444444444445
(0.4, 0.5, 0.4444444444444445, None)
Prediction distribution:
(array([0, 1]), array([22, 10])) (array([0, 1]), array([24,  8]))
[1 1 1 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 1 0 0] [0 0 0 1 1 1 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1]
[1,   501] loss: 0.708
Batch acc = 0.46875 (15.0 / 32)
Batch f1-score = 0.32
(0.26666666666666666, 0.4, 0.32, None)
Prediction distribution:
(array([0, 1]), array([17, 15])) (array([0, 1]), array([22, 10]))
[0 0 0 0 0 1 0 1 0 0 0 1 1 1 1 0 1 1 0 0 0 1 0 1 0 1 1 1 0 1 1 0] [0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 1 1 0 0 0 1 0 0]
[1,  1001] loss: 0.700



Training prediction:
(array([0., 1.]), array([28819, 16423])) (array([0., 1.]), array([32895, 12347]))
[1. 1. 1. 0. 0. 1. 0. 0. 1. 0.] [0. 0. 0. 1. 0. 0. 0. 1. 0. 1.]
F1 of model on train set:  0.3598887730274592
(0.31522864275710893, 0.41929213574147567, 0.3598887730274592, None)
(0.5332172222425677, 0.5387082353733978, 0.5307399920489404, None)


Testing prediction:
(array([0., 1.]), array([7559, 3750])) (array([0., 1.]), array([8223, 3086]))
[1. 0. 0. 0. 0. 0. 0. 1. 0. 0.] [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
F1 of my model on test set:  0.3294324166179052
(0.3002666666666667, 0.3648736228127025, 0.3294324166179052, None)
(0.5204865546589055, 0.5228843366404508, 0.5194874667045931, None)
================== EPOCH INFO: 2 ==================


Batch acc = 0.65625 (21.0 / 32)
Batch f1-score = 0.56
(0.4666666666666667, 0.7, 0.56, None)
Prediction distribution:
(array([0, 1]), array([17, 15])) (array([0, 1]), array([22, 10]))
[0 0 0 1 0 1 1 0 0 0 1 0 1 0 1 1 0 1 1 0 1 1 0 1 0 0 1 0 0 1 1 0] [0 0 0 0 0 1 0 0 1 0 1 1 1 0 1 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0]
[2,   501] loss: 0.698


KeyboardInterrupt: 

In [ ]:
# Eval mode
net.eval()
# Training score
y_pred_train = np.array([])
y_true_train = np.array([])
with torch.no_grad():
    for data_test in tqdm(trainloader):
        images, labels = data_test[0].to(device), data_test[1].to(device)
        outputs = net(images)
        # outputs = torch.round(torch.sigmoid(outputs))
        _, y_pred_tag = torch.max(outputs, 1)
        y_pred_train = np.hstack([y_pred_train, y_pred_tag.cpu().detach().numpy()])

        # for pred_class in outputs.cpu().detach().numpy():
        #     y_pred_train = np.hstack([y_pred_train, pred_class])

        y_true_train = np.hstack([y_true_train, labels.cpu().detach().numpy()])

precision_train, recall_train, f1_train, support_train = precision_recall_fscore_support(y_true_train, y_pred_train, average='binary')
print()
print("Training prediction:")
print(np.unique(y_pred_train, return_counts=True), np.unique(y_true_train, return_counts=True))
print(y_pred_train[0:10], y_true_train[0:10])
print("F1 of model on train set: ", f1_score(y_true_train, y_pred_train))
print("Precision: {}, Recall: {}, F1: {}".format(precision_train, recall_train, f1_train))
np.unique(y_pred_np, return_counts=True), np.unique(y_true_np, return_counts=True)

In [12]:
# Testing score
y_pred_test = np.array([])
y_true_test = np.array([])
with torch.no_grad():
    for data_test in tqdm(testloader):
        images, labels = data_test[0].to(device), data_test[1].to(device)
        outputs = net(images)
        # outputs = torch.round(torch.sigmoid(outputs))

        _, y_pred_tag = torch.max(outputs, 1)
        y_pred_test = np.hstack([y_pred_test, y_pred_tag.cpu().detach().numpy()])

        # for pred_class in outputs.cpu().detach().numpy():
        #     y_pred_test = np.hstack([y_pred_test, pred_class])
            
        y_true_test = np.hstack([y_true_test, labels.cpu().detach().numpy()])
precision_test, recall_test, f1_test, support_test = precision_recall_fscore_support(y_true_test, y_pred_test, average='binary')
print("Accuracy of my model on test set: ", f1_score(y_true_test, y_pred_test))
print("Precision: {}, Recall: {}, F1: {}".format(precision_test, recall_test, f1_test))
np.unique(y_pred_test, return_counts=True), np.unique(y_true_test, return_counts=True)

Accuracy of my model on test set:  0.40753180661577604
Precision: 0.2970767176138893, Recall: 0.6487362281270252, F1: 0.40753180661577604


((array([0., 1.]), array([4570, 6739])),
 (array([0., 1.]), array([8223, 3086])))

In [13]:
torch.save(net.state_dict(), './model/densenet101_{}_{}'.format(f1_train, f1_test))